<a href="https://colab.research.google.com/github/HyeonminNam/BOK4/blob/master/200731_polarity_dic_building_with_NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ngram 감성 사전 구축

## 각 문서에서 필요한 컬럼 추출하여 통합하기

In [ ]:
import pandas as pd
import csv

In [ ]:
df_min = pd.read_csv('df_minutes_15.csv')
df_bond = pd.read_csv('df_bond_15.csv')
df_ed = pd.read_csv('df_eday_15.csv')
df_yh = pd.read_csv('df_yonhap_15.csv')
df_yhi = pd.read_csv('df_yonhapinfo_15.csv')

In [ ]:
df_ed.rename(columns={"callLabel":"call_label"}, inplace = True)

In [ ]:
# 필요한 컬럼만 추출
df_min_new = df_min[['date', 'tokens', 'ngrams', 'call_label']]
df_bond_new = df_bond[['date', 'tokens', 'ngrams', 'call_label']]
df_ed_new = df_ed[['date', 'tokens', 'ngrams', 'call_label']]
df_yh_new = df_yh[['date', 'tokens', 'ngrams', 'call_label']]
df_yhi_new = df_yhi[['date', 'tokens', 'ngrams', 'call_label']]

In [ ]:
# 모든 컬럼 연결
concat_df = pd.concat([df_min_new, df_bond_new, df_ed_new, df_yh_new, df_yhi_new])
concat_df.reset_index(drop=True, inplace=True)

In [ ]:
concat_df

,date,tokens,ngrams,call_label
0,2005-06-09,"['경제/NNG', '경제/NNG', '비해/VV', '더/MAG', '낫/VV',...","['엔달러/NNG;환율/NNG;상승/NNG', '통화/NNG;절상/NNG', '전망...",decrease
1,2005-07-07,"['들/VV', '국제/NNG', '유가/NNG', '크/VA', '오르/VV', ...","['가격/NNG;약세/NNG', '원화/NNG;약세/NNG', '원달러/NNG;급등...",decrease
2,2005-08-11,"['위안/NNG', '절상/NNG', '원화/NNG', '절상/NNG', '가장/M...","['원화/NNG;절상/NNG', '절상/NNG;아니/VCN', 'fed/NNG;금리...",increase
3,2005-09-08,"['금융/NNG', '불안사태/NNG', '발생/NNG', '하/VV', '가운데/...","['경상/NNG;수지/NNG;흑자/NNG', '전망/NNG;인플레이션/NNG;압력/...",decrease
4,2005-10-11,"['금리갭/NNG', '확대/NNG', '되/XSV', '가운데/NNG', '남/V...","['fed/NNG;금리/NNG;인상/NNG', '국채/NNG;발행/NNG;줄/VV'...",increase
...,...,...,...,...
214838,2017-12-29,"['판매/NNG', '넘/VV', '판매/NNG', '증가/NNG', '나타나/VV...","['산업/NNG;긍정적/VAX', '판매/NNG;증가/NNG', '고용/NNG;확대...",increase
214839,2017-12-29,"['다우존스산업평균지수/NNG', '거래일/NNG', '남기/VV', '사상/NNG...","['ff/NNG;금리선물/NNG;시장/NNG;금리/NNG;인상/NNG', '유가/N...",increase
214840,2017-12-29,"['세계/NNG', '금융위기/NNG', '뒤/NNG', '지속/NNG', '금융/...","['기업/NNG;수익/NNG;개선/NNG', '정책/NNG;부작용/NNG', '금리...",increase
214841,2017-12-29,"['고객/NNG', '몰래/MAG', '대출/NNG', '금리/NNG', '인상/N...","['대출/NNG;금리/NNG;인상/NNG', '수익/NNG;오르/VV', '금리/N...",increase


## NBC에 적절한 형식으로 데이터 변환하기

In [ ]:
# token과 ngram 컬럼의 문자열 형식 리스트로 변환하기 위한 함수
def to_lst(text):
    text = text.replace('[',"")
    text = text.replace(']',"")
    text = text.replace('\'',"")
    text = text.split(', ')
    return text

In [ ]:
concat_df['tokens'] = concat_df['tokens'].apply(to_lst)

In [ ]:
concat_df['ngrams'] = concat_df['ngrams'].apply(to_lst)

In [ ]:
# token과 ngram 통합하여 1~5gram 리스트 만들기
concat_df['token+ngram'] = concat_df['tokens'] + concat_df['ngrams']

In [ ]:
concat_df

,date,tokens,ngrams,call_label,token+ngram
0,2005-06-09,"[경제/NNG, 경제/NNG, 비해/VV, 더/MAG, 낫/VV, 특별/NNG, 요...","[엔달러/NNG;환율/NNG;상승/NNG, 통화/NNG;절상/NNG, 전망/NNG;...",decrease,"[경제/NNG, 경제/NNG, 비해/VV, 더/MAG, 낫/VV, 특별/NNG, 요..."
1,2005-07-07,"[들/VV, 국제/NNG, 유가/NNG, 크/VA, 오르/VV, 원화/NNG, 엔/...","[가격/NNG;약세/NNG, 원화/NNG;약세/NNG, 원달러/NNG;급등/NNG,...",decrease,"[들/VV, 국제/NNG, 유가/NNG, 크/VA, 오르/VV, 원화/NNG, 엔/..."
2,2005-08-11,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 크/VA,...","[원화/NNG;절상/NNG, 절상/NNG;아니/VCN, fed/NNG;금리/NNG;...",increase,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 크/VA,..."
3,2005-09-08,"[금융/NNG, 불안사태/NNG, 발생/NNG, 하/VV, 가운데/NNG, 국제/N...","[경상/NNG;수지/NNG;흑자/NNG, 전망/NNG;인플레이션/NNG;압력/NNG...",decrease,"[금융/NNG, 불안사태/NNG, 발생/NNG, 하/VV, 가운데/NNG, 국제/N..."
4,2005-10-11,"[금리갭/NNG, 확대/NNG, 되/XSV, 가운데/NNG, 남/VV, fed/NN...","[fed/NNG;금리/NNG;인상/NNG, 국채/NNG;발행/NNG;줄/VV, 재정...",increase,"[금리갭/NNG, 확대/NNG, 되/XSV, 가운데/NNG, 남/VV, fed/NN..."
...,...,...,...,...,...
214838,2017-12-29,"[판매/NNG, 넘/VV, 판매/NNG, 증가/NNG, 나타나/VV, 보/VV, 현...","[산업/NNG;긍정적/VAX, 판매/NNG;증가/NNG, 고용/NNG;확대/NNG,...",increase,"[판매/NNG, 넘/VV, 판매/NNG, 증가/NNG, 나타나/VV, 보/VV, 현..."
214839,2017-12-29,"[다우존스산업평균지수/NNG, 거래일/NNG, 남기/VV, 사상/NNG, 최고/NN...","[ff/NNG;금리선물/NNG;시장/NNG;금리/NNG;인상/NNG, 유가/NNG;...",increase,"[다우존스산업평균지수/NNG, 거래일/NNG, 남기/VV, 사상/NNG, 최고/NN..."
214840,2017-12-29,"[세계/NNG, 금융위기/NNG, 뒤/NNG, 지속/NNG, 금융/NNG, 완화/N...","[기업/NNG;수익/NNG;개선/NNG, 정책/NNG;부작용/NNG, 금리/NNG;...",increase,"[세계/NNG, 금융위기/NNG, 뒤/NNG, 지속/NNG, 금융/NNG, 완화/N..."
214841,2017-12-29,"[고객/NNG, 몰래/MAG, 대출/NNG, 금리/NNG, 인상/NNG, 혐의/NN...","[대출/NNG;금리/NNG;인상/NNG, 수익/NNG;오르/VV, 금리/NNG;오르...",increase,"[고객/NNG, 몰래/MAG, 대출/NNG, 금리/NNG, 인상/NNG, 혐의/NN..."


In [ ]:
# NBC에 적절한 형식으로 만들기 위해서 띄어쓰기 기준으로 리스트 요소들 결합
def join(lst):
    return ' '.join(lst)

In [ ]:
concat_df['token+ngram'] = concat_df['token+ngram'].apply(join)

In [ ]:
concat_df_new = concat_df[concat_df['call_label'] != '0']

In [ ]:
concat_df_new

,date,tokens,ngrams,call_label,token+ngram
0,2005-06-09,"[경제/NNG, 경제/NNG, 비해/VV, 더/MAG, 낫/VV, 특별/NNG, 요...","[엔달러/NNG;환율/NNG;상승/NNG, 통화/NNG;절상/NNG, 전망/NNG;...",decrease,경제/NNG 경제/NNG 비해/VV 더/MAG 낫/VV 특별/NNG 요인/NNG 없...
1,2005-07-07,"[들/VV, 국제/NNG, 유가/NNG, 크/VA, 오르/VV, 원화/NNG, 엔/...","[가격/NNG;약세/NNG, 원화/NNG;약세/NNG, 원달러/NNG;급등/NNG,...",decrease,들/VV 국제/NNG 유가/NNG 크/VA 오르/VV 원화/NNG 엔/NNG 화간/...
2,2005-08-11,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 크/VA,...","[원화/NNG;절상/NNG, 절상/NNG;아니/VCN, fed/NNG;금리/NNG;...",increase,위안/NNG 절상/NNG 원화/NNG 절상/NNG 가장/MAG 크/VA 추가/NNG...
3,2005-09-08,"[금융/NNG, 불안사태/NNG, 발생/NNG, 하/VV, 가운데/NNG, 국제/N...","[경상/NNG;수지/NNG;흑자/NNG, 전망/NNG;인플레이션/NNG;압력/NNG...",decrease,금융/NNG 불안사태/NNG 발생/NNG 하/VV 가운데/NNG 국제/NNG 환투기...
4,2005-10-11,"[금리갭/NNG, 확대/NNG, 되/XSV, 가운데/NNG, 남/VV, fed/NN...","[fed/NNG;금리/NNG;인상/NNG, 국채/NNG;발행/NNG;줄/VV, 재정...",increase,금리갭/NNG 확대/NNG 되/XSV 가운데/NNG 남/VV fed/NNG 금리/N...
...,...,...,...,...,...
214838,2017-12-29,"[판매/NNG, 넘/VV, 판매/NNG, 증가/NNG, 나타나/VV, 보/VV, 현...","[산업/NNG;긍정적/VAX, 판매/NNG;증가/NNG, 고용/NNG;확대/NNG,...",increase,판매/NNG 넘/VV 판매/NNG 증가/NNG 나타나/VV 보/VV 현지/NNG 업...
214839,2017-12-29,"[다우존스산업평균지수/NNG, 거래일/NNG, 남기/VV, 사상/NNG, 최고/NN...","[ff/NNG;금리선물/NNG;시장/NNG;금리/NNG;인상/NNG, 유가/NNG;...",increase,다우존스산업평균지수/NNG 거래일/NNG 남기/VV 사상/NNG 최고/NNG 기록/...
214840,2017-12-29,"[세계/NNG, 금융위기/NNG, 뒤/NNG, 지속/NNG, 금융/NNG, 완화/N...","[기업/NNG;수익/NNG;개선/NNG, 정책/NNG;부작용/NNG, 금리/NNG;...",increase,세계/NNG 금융위기/NNG 뒤/NNG 지속/NNG 금융/NNG 완화/NNG 벗어나...
214841,2017-12-29,"[고객/NNG, 몰래/MAG, 대출/NNG, 금리/NNG, 인상/NNG, 혐의/NN...","[대출/NNG;금리/NNG;인상/NNG, 수익/NNG;오르/VV, 금리/NNG;오르...",increase,고객/NNG 몰래/MAG 대출/NNG 금리/NNG 인상/NNG 혐의/NNG 법원/N...


In [ ]:
# NBC를 활용한 감성사전 구축 위한 데이터 리스트
tn_lst = list(concat_df_new['token+ngram'])

In [ ]:
tn_lst

['위안부/NNG 합의/NNG 당사자/NNG 적반하장/NNG 불법/NNG 지시/NNG 문/NNG 닫/VV 개성공단/NNG 소상공인/NNG 만들/VV 신문/NNG 개성공단/NNG 중단/NNG 독단/NNG 결정/NNG 과/NNG 산재/NNG 인정/NNG 확대/NNG 장시간/NNG 노동/NNG 경각심/NNG 높/VV 증거/NNG 있/VV 기자/NNG 집단/NNG 폭행/NNG 부인/NNG 적폐/NNG 청산/NNG 외교/NNG 안보/NNG 실타래/NNG 못/MAG 푼다/VV 훈련/NNG 대북/NNG 협상/NNG 안/MAG 되/VV 주한/NNG 미/NNG 사령관/NNG 경고/NNG 국가/NNG 재앙/NNG 모면/NNG 하/XSV 발상/NNG 대전환/NNG 하/VV 판/NNG 개성/NNG 공간/NNG 폐쇄/NNG 시비/NNG 정말/MAG 제정신/NNG 관계/NNG 이렇게/MAG 하/VV 괜찮/VA 위/NNG 아니/VCN 장악/NNG 북핵/NNG 하/VV 파국/NNG 가려/VV 적폐/NNG 청산/NNG 구실/NNG 군/NNG 기밀/NNG 공개/NNG 흐르/VV 안/MAG 되/VV 기자/NNG 폭행/NNG 사건/NNG 제대로/MAG 수사/NNG 하/XSV 한겨레/NNG 위안부/NNG 피해자/NNG 명예/NNG 회복/NNG 되돌아보/VV 개성공단/NNG 폐쇄/NNG 독단/NNG 결정/NNG 보유세/NNG 강화/NNG 부동산/NNG 투기/NNG 공화국/NNG 벗어나/VV 최악/NNG 기부/NNG 한파/NNG 나눔/VV 시민/NNG 의식/NNG 녹이/VV 기자/NNG 집단/NNG 폭행/NNG 어찌/MAG 개인/NNG 일탈/NNG 있/VV 공공기관/NNG 평가/NNG 제도/NNG 개편/NNG 예상/NNG 부작용/NNG 꼭/MAG 막/VV 매일/MAG 경제/NNG 위안부/NNG 합의/NNG 관계/NNG 미래/NNG 균형/NNG 찾/VV 있/VV 규제/NNG 철폐/NNG 경쟁/NNG 하/XSV 韓/NNG 기자/NNG 집단/NNG 폭행/NNG 우발/NNG 단독/NNG 범행/NNG 축소/N

## NBC 활용하여 ngram 감성 사전 구축

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [ ]:
def split_(text):
    lst = text.split(' ')
    return lst

In [ ]:
# 데이터들을 활용하여 벡터화
vectorizer = CountVectorizer(tokenizer=split_)
vectorizer.fit(tn_lst)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
# feature가 될 ngram들의 극성 분류 및 점수 저장할 데이터프레임
dic_df = pd.DataFrame(vectorizer.vocabulary_.keys(), index = vectorizer.vocabulary_.values(), columns = ['tn'])
# 인덱스 기준으로 재정렬
dic_df = dic_df.sort_index(ascending=True)

In [ ]:
# feature들의 빈도 측정
counts = vectorizer.transform(tn_lst)

# 각 문서의 레이블(클래스)
labels = concat_df_new['call_label']
labels = list(labels)

In [ ]:
# 문서의 90%만 선택하여 NBC_classifier 적용.
# 이를 통해서 ngram들의 polarity score 측정(이를 30번 반복하는 bagging 방식 활용)
for i in range(30):
    X_train, X_test, Y_train, Y_test = train_test_split(counts, labels, test_size=0.1)
    classifier = MultinomialNB()
    classifier.fit(X_train, Y_train)
    a = classifier.class_count_
    b = classifier.feature_count_
    score = (b[1]/a[1])/(b[0]/a[0]) # 클수록 hawkish
    dic_df[str(i)] = score

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\ProgramData\Anaconda3\lib\s

In [ ]:
# 30번의 polarity 점수가 데이터프레임에 저장됨
dic_df

,tn,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,3불정책/nng,0.333627,0.305162,0.333546,0.229095,0.304768,0.456796,0.304977,0.304902,0.229129,...,0.229056,0.333302,0.305326,0.332896,0.333595,0.333132,0.304924,0.249465,0.406794,0.166602
1,4대중증질환/nng,1.651455,1.464780,1.651053,1.649484,1.645750,2.055583,2.134836,2.058090,1.649726,...,2.061504,2.138689,1.465565,1.098558,2.064118,1.465780,2.058240,1.646472,1.647516,1.832626
2,4자회담/nng,1.834950,1.830974,1.965539,2.978236,3.047684,1.957698,1.699155,1.829413,2.291285,...,1.963337,3.437179,1.373967,1.830930,2.568680,2.748337,1.437501,1.829413,1.961328,2.290783
3,6자회담/nng,1.440648,1.407585,1.357310,1.433264,1.400833,1.407831,1.407731,1.372060,1.420982,...,1.464009,1.425793,1.377721,1.280192,1.432707,1.396332,1.414951,1.448285,1.433373,1.359631
4,6차산업/nng,2.385435,2.014072,2.384854,2.382588,2.971492,1.644466,3.354742,2.195296,2.382937,...,2.382182,2.199795,3.969239,2.975261,2.385203,2.198669,2.515627,2.972796,2.379745,2.978018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56179,柳/nng,0.733980,0.915487,0.856101,0.916380,0.914305,0.983869,0.792939,0.980043,0.916514,...,0.981668,0.916581,0.854913,1.056306,0.917386,0.850676,0.985141,1.067158,0.915286,0.733051
56180,流水/nng,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
56181,利/nng,0.131068,0.124839,0.087357,0.124961,0.119257,0.124581,0.087136,0.124733,0.087287,...,0.079672,0.119554,0.130854,0.130781,0.083399,0.130873,0.119318,0.083155,0.130755,0.130902
56182,李/nng,1.005118,0.960493,0.997595,0.971951,0.990590,1.032508,1.006877,0.983338,0.973318,...,1.007626,0.975786,0.991372,0.970415,1.010698,0.980362,0.951100,0.981311,0.990455,0.987139


In [ ]:
import numpy as np

In [ ]:
# 평균 점수를 저장할 컬럼
dic_df['polarity_score'] = 0.0

# 평균 점수 저장
for i, row in dic_df.iterrows():
    mean = np.mean(dic_df.iloc[i][1:31])
    dic_df['polarity_score'][i] = mean

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# ngram 목록과 극성 점수만 다시 추출
dic_df = dic_df[['tn', 'polarity_score']]

In [ ]:
# 극성 점수 기반으로 극성 판단. 기본은 neutral.
# 1/1.3 미만이면 dovish, 1.3보다 크면 hawkish. 나머지는 gray area에 들어가서 neutral.
dic_df['label'] = 'neutral'
dic_df.loc[dic_df['polarity_score'] > 1.3, 'label'] = 'hawkish'
dic_df.loc[dic_df['polarity_score'] < 1/1.3, 'label'] = 'dovish'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dic_df

,tn,polarity_score,label
0,3불정책/nng,0.306038,dovish
1,4대중증질환/nng,1.760403,hawkish
2,4자회담/nng,2.069138,hawkish
3,6자회담/nng,1.394971,hawkish
4,6차산업/nng,2.522239,hawkish
...,...,...,...
56179,柳/nng,0.921028,neutral
56180,流水/nng,0.000000,dovish
56181,利/nng,0.114255,dovish
56182,李/nng,0.991500,neutral


In [ ]:
# hawkish인 단어 15,786개
dic_df[dic_df['label']=='hawkish']

,tn,polarity_score,label
1,4대중증질환/nng,1.760403,hawkish
2,4자회담/nng,2.069138,hawkish
3,6자회담/nng,1.394971,hawkish
4,6차산업/nng,2.522239,hawkish
5,a+등급/nng,1.449664,hawkish
...,...,...,...
56143,힘든지/va,inf,hawkish
56161,魯/nng,2.206424,hawkish
56163,不/nng,1.783124,hawkish
56170,女/nng,1.570148,hawkish


In [ ]:
# dovish인 단어 개 18,154개
dic_df[dic_df['label']=='dovish']

NameError: name 'dic_df' is not defined

In [ ]:
# csv 파일로 저장
dic_df.to_csv('polarity_dic.csv', encoding='utf-8-sig', index=False)